# Predict the road sign Neural Network Approach
## ( Keras Framework with Tensorflow backend)

In [1]:
## importing the pre-requisites for data handling 

import numpy as np
import pandas as pd

## importing pre-requisites for building Neural Net 

from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from sklearn.preprocessing import StandardScaler,LabelEncoder
from keras.utils.np_utils import to_categorical as tocat

Using TensorFlow backend.


In [2]:
# to initialize psuedorandom number generator (can work as key to generate same results)

np.random.seed(10)

In [3]:
# lets load the datsets

train=pd.read_csv('train.csv') #training 
test=pd.read_csv('test.csv') # real Testing!

In [4]:
# encoding string values to 0,1,2,3  and storing Actuals in y_train

le=LabelEncoder().fit(train.DetectedCamera)
y_train=le.transform(train['SignFacing (Target)'])

In [5]:
# renaming llooooonnngg column names

train.rename(columns={'SignFacing (Target)': 'Actual', 'DetectedCamera': 'Detected'}, inplace=True)

In [7]:
# removing useless columns(Id,Actual) and encoding Detected camera value to 0,1,2,3

X_train = train.drop(['Id','Actual'],axis=1)
X_train['Detected'] = le.transform(X_train['Detected'])
X_train.head()

,Detected,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,2,195,1.02,46,45
1,2,203,1.09,59,54
2,0,26,0.96,104,108
3,2,199,0.81,38,47
4,2,208,0.93,54,58


In [8]:
# scaling the numerical columns thus obtained to improve prediction results
## converting y_train to One-Hot Encoding for probablistic values 

scaler=StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
y_train = tocat(y_train)

In [97]:
# Let's have a look at the test data for the first time 

test.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,2c9180975a056a64015a1e10d3f270fe,Right,67,0.63,107,169
1,2c9180975a056a64015a1de4deb16bdc,Front,16,0.88,61,69
2,2c9180975a056a64015a1e0e70ea70ce,Right,44,1.15,232,202
3,2c9180975a056a64015a1dfed0c46ec6,Right,50,1.10,137,125
4,2c9180975a056a64015a1dfed0c46ec7,Front,30,0.95,99,104


In [9]:
# Removing useless columns(Id) and transforming Detected values to corresponding numerical counterparts 0,1,2,3

Id = test.Id
test['DetectedCamera'] = le.transform(test['DetectedCamera'])
X_test = test.drop(['Id'],axis=1)
X_test.shape

(31485, 5)

In [10]:
# Lets scale the test data now to proportion for achieving better results

X_test=scaler.transform(X_test)

# Neural Network Build
## A 4 layer model 
### Layer1 : 512 Neurons, and input weights equal to number of features, Activation -> ReLU
### Layer2 : 256 Neurons, Activation -> ReLU
### Layer3 : 64 Neurons, Activation -> sigmoid
### Output Layer : 4 Outputs, Activation -> softmax (used for output layer) 

In [11]:
#model begins


model = Sequential()

In [12]:
#layer 1

model.add(Dense(512,input_dim=5))
model.add(Activation("relu"))
model.add(Dropout(0.1))


In [13]:
#layer 2

model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.2))

In [14]:
#layer 2

model.add(Dense(64))
model.add(Activation("sigmoid"))
model.add(Dropout(0.2))

In [15]:
#layer 4

model.add(Dense(4))
model.add(Activation("softmax"))

In [16]:
## compiling the model

### optimizer used ins rmsprop (adagrad is also an optimizer)
### since we are working in multinomial classification problem loss must be calculated using categorical_crossentropy

model.compile(optimizer = "rmsprop",loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
#lets run the model built

## iterations = 100
## each iteration batch_size is 100
## set verbose=1 for checking the training status


params = model.fit(X_train, y_train, epochs=100, batch_size=100, verbose=0)


In [18]:
# Predict the results for this learned model

y_test = model.predict_proba(X_test)

31200/31485 [============================>.] - ETA: 0s 

In [19]:
# submit the results in submissionNN.csv
## Job Done!

submission=pd.DataFrame(y_test,index=Id,columns=le.classes_)
submission.to_csv('submission_NN.csv')